In [1]:
## all imports
from IPython.display import HTML
import numpy as np
from urllib.request import urlopen
import requests
from urllib.error import HTTPError, URLError
import bs4 #this is beautiful soup
import time
import operator
import socket
import pickle
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

In [2]:
## Initialise DataFrame

df_data_list = []
df_data_list

[]

In [3]:
###### Step 1: Get the total number of pages available in site ###
url = "https://www.tripadvisor.com.my/Restaurants-g294229-c8-Jakarta_Java.html"
source = requests.get(url)
# parse html code
site_html = bs4.BeautifulSoup(source.text, 'html.parser')
print(site_html)

<!DOCTYPE html>

<html lang="en" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<link data-rup="long_lived_global_legacy" href="https://static.tacdn.com/css2/build/concat/long_lived_global_legacy-v22732983036b.css" rel="stylesheet" type="text/css"/>
<link href="https://static.tacdn.com/favicon.ico" id="favicon" rel="icon" type="image/x-icon"/>
<link as="font" crossorigin="" href="https://static.tacdn.com/css2/webfonts/TripAdvisor/TripAdvisor_Regular.woff2?v004.000" rel="preload" type="font/woff2"/>
<link color="#00a680" href="https://static.tacdn.com/img2/icons/ta_square.svg" rel="mask-icon" sizes="any">
<script data-rup="global_error" type="text/javascript">!function(){function e(e){return"object"==typeof e&&"WARN"===e.level?"WARN":"ERROR"}function r(r,n,o,i,a,c){var d={error_script:n,line:o,column:i,ready_state:document.readyState};return s?(require.defined("ta/util/Error")&&require("ta/util/Error").record(a,"

In [4]:
## Search div tags for pageNumbers class

div_pageNumber = site_html.find('div', class_='pageNumbers')
div_pageNumber

<div class="pageNumbers">
<span class="pageNum current" data-offset="0" data-page-number="1" onclick="ta.trackEventOnPage('STANDARD_PAGINATION', 'curpage', '1', 0)">1</span>
<a class="pageNum taLnk" data-offset="30" data-page-number="2" href="/Restaurants-g294229-c8-oa30-Jakarta_Java.html#EATERY_LIST_CONTENTS" onclick="      ta.restaurant_filter.paginate(this.getAttribute('data-offset'));; ta.trackEventOnPage('STANDARD_PAGINATION', 'page', '2', 0); return false;
  ">
2
</a>
<a class="pageNum taLnk" data-offset="60" data-page-number="3" href="/Restaurants-g294229-c8-oa60-Jakarta_Java.html#EATERY_LIST_CONTENTS" onclick="      ta.restaurant_filter.paginate(this.getAttribute('data-offset'));; ta.trackEventOnPage('STANDARD_PAGINATION', 'page', '3', 0); return false;
  ">
3
</a>
<a class="pageNum taLnk" data-offset="90" data-page-number="4" href="/Restaurants-g294229-c8-oa90-Jakarta_Java.html#EATERY_LIST_CONTENTS" onclick="      ta.restaurant_filter.paginate(this.getAttribute('data-offset'))

In [11]:
## Get total page numbers
cafe_list_href = []
page_number_total = int(div_pageNumber.contents[-2].get('data-page-number'))
page_number_total

42

In [14]:
### Step 2 Extract all TA_Link to each cafe
# Each page shows 30 results
query_string_geo = 294229


for pn in [41,38,29,30,31,7,0]:
    print('Scanning page ', pn + 1)
    query_string_o = 'a' + str(pn*30)
    query_string = {
            'Action': 'PAGE',
            'geo': query_string_geo,
            'ajax': 1,
            'cat': 10642,
            'sortOrder': 'ranking',
            'o': query_string_o,
            'availSearchEnabled': 'false'
        }
    
    while (True):
        try:
            page = requests.get('https://www.tripadvisor.com.my/RestaurantSearch', params = query_string)
        except HTTPError:
            print("HTTPError:")
            continue
        except URLError:
            print("URLError:")
            continue
        except socket.error as error:
            print("Connection closed")
            continue
        break

    page_html = bs4.BeautifulSoup(page.text,'html.parser')
    #print(page_html.prettify())
    #print('\n')
    #print('\n')
    
    cafe_list_page_a = page_html.findAll('a', class_='restaurants-list-ListCell__restaurantName--2aSdo')
    #print(cafe_list_page_a)
    cafe_list_page_href = [ref.get('href') for ref in cafe_list_page_a]
    print(cafe_list_page_href)
    print('\n')
    for item in cafe_list_page_href:
        cafe_list_href.append(item)

Scanning page  42
['/Restaurant_Review-g294229-d17838662-Reviews-Umauma_Eatery_Shop-Jakarta_Java.html']


Scanning page  39
['/Restaurant_Review-g294229-d7707441-Reviews-Bakoel_Koffie-Jakarta_Java.html', '/Restaurant_Review-g294229-d7707448-Reviews-Edam_Burger_Cafe-Jakarta_Java.html', '/Restaurant_Review-g294229-d7709381-Reviews-Bubble_Bude-Jakarta_Java.html', '/Restaurant_Review-g294229-d7709383-Reviews-QBox_Cafe-Jakarta_Java.html', '/Restaurant_Review-g294229-d7712111-Reviews-Dante_Coffee-Jakarta_Java.html', '/Restaurant_Review-g294229-d7712327-Reviews-Halte_Roti_Panggang-Jakarta_Java.html', '/Restaurant_Review-g294229-d7712423-Reviews-The_Coffee_Bean_Tea_Leaf-Jakarta_Java.html', '/Restaurant_Review-g294229-d7712435-Reviews-Kedai_Kopi_Soeboer-Jakarta_Java.html', '/Restaurant_Review-g294229-d7712496-Reviews-Happy_Bubble-Jakarta_Java.html', '/Restaurant_Review-g294229-d7712543-Reviews-Air_Mata_Kucing-Jakarta_Java.html', '/Restaurant_Review-g294229-d7712558-Reviews-Warkop_Garuda-Jakarta

['/Restaurant_Review-g294229-d1094825-Reviews-Victoria_Cafe-Jakarta_Java.html', '/Restaurant_Review-g294229-d6488322-Reviews-Starbucks-Jakarta_Java.html', '/Restaurant_Review-g294229-d15661623-Reviews-Kopi_Kenangan-Jakarta_Java.html', '/Restaurant_Review-g294229-d6720440-Reviews-Dunkin_Donuts_Kelapa_Gading-Jakarta_Java.html', '/Restaurant_Review-g294229-d10392890-Reviews-Lulu_Lala-Jakarta_Java.html', '/Restaurant_Review-g294229-d10597021-Reviews-Kuppie_Coffee-Jakarta_Java.html', '/Restaurant_Review-g294229-d10523406-Reviews-Homebound_Coffee-Jakarta_Java.html', '/Restaurant_Review-g294229-d6489787-Reviews-Starbucks-Jakarta_Java.html', '/Restaurant_Review-g294229-d8398454-Reviews-The_Cat_Cabin-Jakarta_Java.html', '/Restaurant_Review-g294229-d13428687-Reviews-Gelato_Secrets-Jakarta_Java.html', '/Restaurant_Review-g294229-d6173499-Reviews-Cheesecake_Factory_Bintaro-Jakarta_Java.html', '/Restaurant_Review-g294229-d5552861-Reviews-Cafe_Oh_la_la-Jakarta_Java.html', '/Restaurant_Review-g294229

In [15]:
print(len(cafe_list_href))

1171


In [16]:
df_data_list = []

In [17]:
### Step 3 Extract info from each cafe TA_Link and append to DataFrame

base_url = 'https://www.tripadvisor.com.my/'
link_crawl_counter = 0

for TA_link_href in cafe_list_href:
    print(link_crawl_counter)
    TA_link = base_url + TA_link_href
    try:
        TA_link_page = requests.get(TA_link)
    except HTTPError:
        print("HTTPError:")
        continue
    except URLError:
        print("URLError:")
        continue
    except socket.error as error:
        print("Connection closed")
        continue
    TA_link_html = bs4.BeautifulSoup(TA_link_page.text,'html.parser')
    #print(TA_link_html.prettify())
    
    title = None
    address = None
    email = None
    tel = None

    #Title data
    title_class = TA_link_html.find('h1', class_='ui_header h1')
    if title_class is not None:
        title = title_class.contents[0]
    
    loccon = TA_link_html.find('div', class_='restaurants-detail-overview-cards-LocationOverviewCard__cardColumn--2ALwF')
    if loccon is not None:
        #Address Data
        location = loccon.find('div', class_='restaurants-detail-overview-cards-LocationOverviewCard__addressLink--1pLK4 restaurants-detail-overview-cards-LocationOverviewCard__detailLink--iyzJI')
        if location is not None:
            location_ui_link = location.find('div', class_='ui_link')
            if location_ui_link is not None:
                address = location_ui_link.find('span', class_='restaurants-detail-overview-cards-LocationOverviewCard__detailLinkText--co3ei').contents[0]    
    
        #Email
        mailto = loccon.find(href=re.compile("mailto"))
        if mailto is not None:
            mailto_href = mailto.get('href')
            email_re = re.search('mailto:(.+?)\?subject=\?', mailto_href)
            if email_re:
                email = email_re.group(1)
        
        #Call
        tel = loccon.find(href=re.compile("tel:"))
        if tel is not None:
            tel_href = tel.get('href')
            tel_re = re.search('tel:(.*)', tel_href)
            if tel_re:
                tel = tel_re.group(1)
    
    df_data_element = {
        'Name': title,
        'TA_Link': TA_link,
        'Contact': tel,
        'Email': email,
        'Address': address
    }
    df_data_list.append(df_data_element)
    link_crawl_counter += 1

df_data = pd.DataFrame(df_data_list)
df_data


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,Address,Contact,Email,Name,TA_Link
0,"Jl. Pluit Sakti Raya no.117 Muara Karang, Jaka...",+62 811-160-226,None,Ombe Kofie,https://www.tripadvisor.com.my//Restaurant_Rev...
1,"Jl. M.H. Thamrin no. 1 Grand Indonesia 8F, CGV...",+62 21 29923929,touslesjours-id@cj.co.id,Tous Les Jours Grand Indonesia,https://www.tripadvisor.com.my//Restaurant_Rev...
2,"Jl. Suryo No. 25 Senopati, Jakarta 12180 Indon...",+62 21 72780012,getcrematology@gmail.com,Crematology X Senopati,https://www.tripadvisor.com.my//Restaurant_Rev...
3,Kav. 3-5 Jl. Prof. Dr. Satrio Lotte Shopping A...,+62 21 29889166,None,Blacklisted,https://www.tripadvisor.com.my//Restaurant_Rev...
4,Jln. Melawai VI no. 25 Kawasan Terpadu Blok M ...,None,None,"Kedai Filosofi Kopi, Melawai",https://www.tripadvisor.com.my//Restaurant_Rev...
5,Jln. Pintu Besar Utara no. 11 Kota Tua Distric...,+62 21 6904188,historiajakarta@gmail.com,Historia Food & Bar,https://www.tripadvisor.com.my//Restaurant_Rev...
6,Jln. M.H. Thamrin No. 1 Grand Indonesia 2 Kino...,+62 21 23580152,None,Starbucks - Grand Indonesia,https://www.tripadvisor.com.my//Restaurant_Rev...
7,"Jln. Rawamangun Muka Barat Selatan VI, Jakarta...",+62 622 14892640,locale@ymail.com,Kedai Locale,https://www.tripadvisor.com.my//Restaurant_Rev...
8,"Jln. Teuku Cikditiro No. 52, Jakarta 10310 Ind...",+62 622 13106370,None,Anomali Coffee,https://www.tripadvisor.com.my//Restaurant_Rev...
9,"Taman Ratu Indah blok AA2 no 33, Jakarta 11520...",+62 21 29520310,info@koultouracoffee.com,Koultoura Coffee,https://www.tripadvisor.com.my//Restaurant_Rev...


In [18]:
df_data.shape

(1171, 5)

In [19]:
df_data.to_csv('jakartav2.csv')